In [1]:
import gym
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import rl_utils
from datetime import datetime

In [2]:
from SheepDogEnv import SheepDogEnv


def sample_expert_data(n_episode):
    states = []
    actions = []
    with tqdm(total=n_episode, desc="进度条") as pbar:
        for i in range(n_episode):
            done = False
            while not done:
                _st = env._get_obs_array()
                L1 = _st[0]
                L2 = env.circle_R
                theta1 = env.sheep_polar_coor[1]
                theta2 = env.dog_theta[0]
                if(L1**2 + L2**2 - 2*L1*L2*np.cos(theta2-theta1)) < 0:
                    break
                L3 = np.sqrt(L1**2 + L2**2 - 2*L1*L2*np.cos(theta2-theta1))
                if(L3 == 0):
                    break
                theta3 = np.arcsin(L2/L3*np.sin(theta2-theta1))
                if(np.abs(theta2-theta1) < np.arccos(L1/L2)):
                    theta3 = np.pi/2 if theta3 > 0 else -np.pi/2
                action = theta3

                # if(np.abs(_st[3]/env.dog_theta_v) > (env.circle_R-_st[0])/env.sheep_v + 1):
                #     action = 0

                _st[0] /= env.circle_R
                states.append(_st)
                actions.append(action)
                _, _, done, _, _ = env.step(action)
                # print(i, _st, action)
            pbar.update(1)
            env.reset()
            env.sheep_polar_coor = np.array(
                [np.random.randint(env.sheep_v, env.circle_R-1), np.random.random()*np.pi*2])
            env.dog_theta = np.array(
                [env.sheep_polar_coor[1]+np.pi/2+np.pi*np.random.random()])
    return np.array(states), np.array(actions)


env = SheepDogEnv(circle_R=350, sheep_v=30, dog_v=80,
                  sec_split_n=5, store_mode=False, render_mode=False)

random.seed(0)
n_episode = 20000
expert_s, expert_a = sample_expert_data(n_episode)

print(len(expert_s))
# n_samples = 30  # 采样30个数据
# random_index = random.sample(range(expert_s.shape[0]), n_samples)
# expert_s = expert_s[random_index]
# expert_a = expert_a[random_index]


进度条: 100%|██████████| 20000/20000 [02:34<00:00, 129.08it/s]


3889677


In [55]:
# index = expert_s[:,3]!=0

# print(len(expert_s))
# expert_a=expert_a[index]
# expert_s=expert_s[index]
print(len(expert_s))

np.save("expert_a-{}.npy".format(str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))),expert_a)
np.save("expert_s-{}.npy".format(str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))),expert_s)

1135509


In [38]:
import os

def list_files_with_prefix(directory, prefix):
    file_list = [file for file in os.listdir(directory) if file.startswith(prefix)]
    return file_list

expert_s_files = list_files_with_prefix('./', 'expert_s')
expert_a_files = list_files_with_prefix('./', 'expert_a')

# expert_s=np.array([[0,0,0,0]])
# expert_a=np.array([])
for expert_s_file in  expert_s_files:
    tmp_expert_s=np.load(expert_s_file)
    expert_s=np.concatenate((expert_s,tmp_expert_s),axis=0)
# expert_s=expert_s[1:]
for expert_a_file in  expert_a_files:
    tmp_expert_a=np.load(expert_a_file)
    expert_a=np.concatenate((expert_a,tmp_expert_a),axis=0)

print(len(expert_s),len(expert_a))

10388787 10388788


In [21]:
expert_s=expert_s[:,[0,3]]

In [3]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = np.pi/2*torch.tanh(self.fc3(x))
        return x
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

In [22]:
import time


class BehaviorClone:
    def __init__(self, state_dim, hidden_dim, action_dim, lr):
        self.policy = PolicyNet(state_dim, hidden_dim, action_dim).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr)

    def learn(self, states, actions):
        states = torch.tensor(states, dtype=torch.float).to(device)
        actions = torch.tensor(
            actions, dtype=torch.float).view(-1, 1).to(device)
        mse_loss = F.mse_loss(self.policy(states), actions).float()
        # print(mse_loss)

        self.optimizer.zero_grad()
        mse_loss.backward()
        self.optimizer.step()

    def take_action(self, state):
        state = torch.tensor([state], dtype=torch.float).to(device)
        action = self.policy(state).cpu().detach().numpy()
        return action[0]


def test_agent(agent, env, n_episode):
    sheep_coor_test = [0, 1.7, np.pi, 4.5, 2*np.pi-0.1]
    dog_theta_test = [0, 1.7, np.pi, 4.5, 2*np.pi-0.1]
    specific_return_list = []
    catched_num = 0
    catch_init_state_list=[]
    for s in sheep_coor_test:
        for d in dog_theta_test:
            env.reset()
            env.sheep_polar_coor = np.array([env.sheep_v, s])
            env.dog_theta = np.array([d])
            episode_return = 0
            for i in range(2000):
                _st = env._get_obs_array()
                _st[0] /= env.circle_R
                action = agent.take_action(_st[[0,3]])[0]
                _, reward, done, _, _ = env.step(action)
                episode_return += reward
                # print(_st, action, reward)
                if done:
                    break
            specific_return_list.append(episode_return)
            # print("s:{},d:{},最终得分：{}".format(s, d, episode_return))
            if reward < -900:
                catched_num += 1
                catch_init_state_list.append((s,d))
    print("sheep has been catched: {}/{},init_states: {}".format(len(sheep_coor_test)
          * len(dog_theta_test), catched_num,",".join([str(i) for i in catch_init_state_list])))
    if(catched_num==0):
        torch.save(agent.policy.state_dict(),"bc-"+str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S")))
    return np.mean(specific_return_list)


torch.manual_seed(0)
np.random.seed(0)

lr = 5e-6
env = SheepDogEnv(circle_R=350, sheep_v=30, dog_v=80,
                  sec_split_n=5, store_mode=False, render_mode=False)
bc_agent = BehaviorClone(
    2, 128, env.action_space.shape[0], lr)
n_iterations = 20000
batch_size = 512
test_returns = []

with tqdm(total=n_iterations, desc="进度条") as pbar:
    for i in range(n_iterations):
        sample_indices = np.random.randint(low=0,
                                           high=expert_s.shape[0],
                                           size=batch_size)
        t1 = time.time()
        bc_agent.learn(expert_s[sample_indices], expert_a[sample_indices])
        if i > 9000 and (i+1) % 100 == 0:
            current_return = test_agent(bc_agent, env, 5)
            test_returns.append(current_return)
            # if (i + 1) % 100 == 0:
            #     pbar.set_postfix(
            #         {'return': '%.3f' % np.mean(test_returns[-10:])})
        pbar.update(1)


进度条:  45%|████▌     | 9098/20000 [00:13<00:17, 638.13it/s]C:\Users\77016\AppData\Local\Temp\ipykernel_24300\3588853854.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:204.)
  state = torch.tensor([state], dtype=torch.float).to(device)
进度条:  46%|████▌     | 9162/20000 [00:14<00:58, 183.85it/s]

sheep has been catched: 25/0,init_states: 


进度条:  46%|████▋     | 9262/20000 [00:15<01:14, 143.64it/s]

sheep has been catched: 25/0,init_states: 


进度条:  47%|████▋     | 9361/20000 [00:16<01:23, 127.17it/s]

...

sheep has been catched: 25/13,init_states: (0, 0),(0, 4.5),(0, 6.183185307179587),(1.7, 0),(1.7, 1.7),(1.7, 6.183185307179587),(3.141592653589793, 1.7),(3.141592653589793, 3.141592653589793),(4.5, 3.141592653589793),(4.5, 4.5),(6.183185307179587, 0),(6.183185307179587, 4.5),(6.183185307179587, 6.183185307179587)


进度条:  80%|███████▉  | 15954/20000 [01:28<00:43, 93.55it/s]

sheep has been catched: 25/13,init_states: (0, 0),(0, 4.5),(0, 6.183185307179587),(1.7, 0),(1.7, 1.7),(1.7, 6.183185307179587),(3.141592653589793, 1.7),(3.141592653589793, 3.141592653589793),(4.5, 3.141592653589793),(4.5, 4.5),(6.183185307179587, 0),(6.183185307179587, 4.5),(6.183185307179587, 6.183185307179587)


进度条:  80%|███████▉  | 15999/20000 [01:28<00:22, 180.99it/s]


KeyboardInterrupt: 

In [6]:
def test_agent(agent, env, n_episode):
    sheep_coor_test = [0, 1.7, np.pi, 4.5, 2*np.pi-0.1]
    dog_theta_test = [0, 1.7, np.pi, 4.5, 2*np.pi-0.1]
    specific_return_list = []
    catched_num = 0
    catch_init_state_list=[]
    for s in sheep_coor_test:
        for d in dog_theta_test:
            env.reset()
            env.sheep_polar_coor = np.array([env.sheep_v, s])
            env.dog_theta = np.array([d])
            episode_return = 0
            for i in range(2000):
                _st = env._get_obs_array()
                _st[0] /= env.circle_R
                action = agent.take_action(_st)[0]
                _, reward, done, _, _ = env.step(action)
                episode_return += reward
                # print(_st, action, reward)
                if done:
                    break
            specific_return_list.append(episode_return)
            print("s:{},d:{},最终得分：{}".format(s, d, episode_return))
            if reward < -900:
                catched_num += 1
                catch_init_state_list.append((s,d))
    print("sheep has been catched: {}/{},init_states: {}".format(len(sheep_coor_test)
          * len(dog_theta_test), catched_num,",".join([str(i) for i in catch_init_state_list])))
    return np.mean(specific_return_list)


test_agent(bc_agent, env, 5)


s:0,d:0,最终得分：-1324.328041493539
s:0,d:1.7,最终得分：613.714447679481
s:0,d:3.141592653589793,最终得分：755.8463644482215
s:0,d:4.5,最终得分：774.964312341708
s:0,d:6.183185307179587,最终得分：756.8657026697047
s:1.7,d:0,最终得分：773.0200719665925
s:1.7,d:1.7,最终得分：732.752254705843
s:1.7,d:3.141592653589793,最终得分：632.8096865214945
s:1.7,d:4.5,最终得分：777.4381304789692
s:1.7,d:6.183185307179587,最终得分：771.7153007977226
s:3.141592653589793,d:0,最终得分：771.8395152019557
s:3.141592653589793,d:1.7,最终得分：746.3676194663918
s:3.141592653589793,d:3.141592653589793,最终得分：772.5737135569807
s:3.141592653589793,d:4.5,最终得分：744.5173100029378
s:3.141592653589793,d:6.183185307179587,最终得分：771.821691564696
s:4.5,d:0,最终得分：770.7485704354729
s:4.5,d:1.7,最终得分：751.5368815301912
s:4.5,d:3.141592653589793,最终得分：762.3170737973666
s:4.5,d:4.5,最终得分：766.9470525297797
s:4.5,d:6.183185307179587,最终得分：770.2244192137242
s:6.183185307179587,d:0,最终得分：-1317.9678791263775
s:6.183185307179587,d:1.7,最终得分：484.1831442943279
s:6.183185307179587,d:3.141592653589793,最

573.9550423604549

In [15]:
torch.save(bc_agent.policy.state_dict(),"bc_network-1")

In [80]:
from SheepDogEnv import SheepDogEnv


def sample_expert_data(n_episode):
    states = []
    actions = []
    with tqdm(total=n_episode, desc="进度条") as pbar:
        for i in range(n_episode):
            done = False
            while not done:
                _st = env._get_obs_array()
                L1 = _st[0]
                L2 = env.circle_R
                theta1 = _st[1]
                theta2 = _st[2]
                if(L1**2 + L2**2 - 2*L1*L2*np.cos(theta2-theta1))<0:
                    break
                L3 = np.sqrt(L1**2 + L2**2 - 2*L1*L2*np.cos(theta2-theta1))
                if(L3==0):
                    break
                theta3 = np.arcsin(L2/L3*np.sin(theta2-theta1))
                if(np.abs(theta2-theta1) < np.arccos(L1/L2)):
                    theta3 = np.pi/2 if theta3 > 0 else -np.pi/2
                action = theta3

                # if(np.abs(_st[3]/env.dog_theta_v) > (env.circle_R-_st[0])/env.sheep_v + 1):
                #     action = 0

                _st[0] /= env.circle_R
                states.append(_st)
                actions.append(action)
                _, _, done, _, _ = env.step(action)
                # print(i, _st, action)
            pbar.update(1)
            env.reset()
            env.sheep_polar_coor = np.array(
                [np.random.randint(env.sheep_v, env.circle_R-1), 0.0])
            env.dog_theta = np.array([0])
    return np.array(states), np.array(actions)


env = SheepDogEnv(circle_R=350, sheep_v=30, dog_v=80,
                  sec_split_n=5, store_mode=False, render_mode=False)

random.seed(0)
n_episode = 500
expert_s, expert_a = sample_expert_data(n_episode)

print(len(expert_s))
# n_samples = 30  # 采样30个数据
# random_index = random.sample(range(expert_s.shape[0]), n_samples)
# expert_s = expert_s[random_index]
# expert_a = expert_a[random_index]


进度条: 100%|██████████| 500/500 [00:29<00:00, 17.03it/s]


743871


In [81]:
import time


class BehaviorClone:
    def __init__(self, state_dim, hidden_dim, action_dim, lr):
        self.policy = PolicyNet(state_dim, hidden_dim, action_dim).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr)

    def learn(self, states, actions):
        states = torch.tensor(states, dtype=torch.float).to(device)
        actions = torch.tensor(
            actions, dtype=torch.float).view(-1, 1).to(device)
        mse_loss = F.mse_loss(self.policy(states), actions).float()
        # print(mse_loss)

        self.optimizer.zero_grad()
        mse_loss.backward()
        self.optimizer.step()

    def take_action(self, state):
        state = torch.tensor([state], dtype=torch.float).to(device)
        action = self.policy(state).cpu().detach().numpy()
        return action[0]


def test_agent(agent, env, n_episode):
    sheep_coor_test = [0, 1.7, np.pi, 4.5, 2*np.pi-0.1]
    dog_theta_test = [0, 1.7, np.pi, 4.5, 2*np.pi-0.1]
    specific_return_list = []
    catched_num = 0
    catch_init_state_list=[]
    for s in sheep_coor_test:
        for d in dog_theta_test:
            env.reset()
            env.sheep_polar_coor = np.array([env.sheep_v, s])
            env.dog_theta = np.array([d])
            episode_return = 0
            for i in range(2000):
                _st = env._get_obs_array()
                _st[0] /= env.circle_R
                action = agent.take_action(_st)[0]
                _, reward, done, _, _ = env.step(action)
                episode_return += reward
                # print(_st, action, reward)
                if done:
                    break
            specific_return_list.append(episode_return)
            # print("s:{},d:{},最终得分：{}".format(s, d, episode_return))
            if reward < -900:
                catched_num += 1
                catch_init_state_list.append((s,d))
    print("sheep has been catched: {}/{},init_states: {}".format(len(sheep_coor_test)
          * len(dog_theta_test), catched_num,",".join([str(i) for i in catch_init_state_list])))
    if(catched_num==0):
        torch.save(agent.policy.state_dict(),"bc-"+str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S")))
    return np.mean(specific_return_list)


torch.manual_seed(0)
np.random.seed(0)

lr = 5e-6
env = SheepDogEnv(circle_R=350, sheep_v=30, dog_v=80,
                  sec_split_n=5, store_mode=False, render_mode=False)
# bc_agent = BehaviorClone(
#     env.observation_space.shape[0], 128, env.action_space.shape[0], lr)
n_iterations = 2000
batch_size = 512
test_returns = []

with tqdm(total=n_iterations, desc="进度条") as pbar:
    for i in range(n_iterations):
        sample_indices = np.random.randint(low=0,
                                           high=expert_s.shape[0],
                                           size=batch_size)
        t1 = time.time()
        bc_agent.learn(expert_s[sample_indices], expert_a[sample_indices])
        if i > 900 and (i+1) % 200 == 0:
            current_return = test_agent(bc_agent, env, 5)
            test_returns.append(current_return)
            # if (i + 1) % 100 == 0:
            #     pbar.set_postfix(
            #         {'return': '%.3f' % np.mean(test_returns[-10:])})
        # pbar.update(1)


进度条:   0%|          | 0/2000 [00:00<?, ?it/s]

sheep has been catched: 25/25,init_states: (0, 0),(0, 1.7),(0, 3.141592653589793),(0, 4.5),(0, 6.183185307179587),(1.7, 0),(1.7, 1.7),(1.7, 3.141592653589793),(1.7, 4.5),(1.7, 6.183185307179587),(3.141592653589793, 0),(3.141592653589793, 1.7),(3.141592653589793, 3.141592653589793),(3.141592653589793, 4.5),(3.141592653589793, 6.183185307179587),(4.5, 0),(4.5, 1.7),(4.5, 3.141592653589793),(4.5, 4.5),(4.5, 6.183185307179587),(6.183185307179587, 0),(6.183185307179587, 1.7),(6.183185307179587, 3.141592653589793),(6.183185307179587, 4.5),(6.183185307179587, 6.183185307179587)
sheep has been catched: 25/24,init_states: (0, 0),(0, 1.7),(0, 3.141592653589793),(0, 4.5),(0, 6.183185307179587),(1.7, 0),(1.7, 1.7),(1.7, 3.141592653589793),(1.7, 4.5),(1.7, 6.183185307179587),(3.141592653589793, 0),(3.141592653589793, 1.7),(3.141592653589793, 3.141592653589793),(3.141592653589793, 4.5),(3.141592653589793, 6.183185307179587),(4.5, 0),(4.5, 3.141592653589793),(4.5, 4.5),(4.5, 6.183185307179587),(6.183

进度条:   0%|          | 0/2000 [00:30<?, ?it/s]

sheep has been catched: 25/25,init_states: (0, 0),(0, 1.7),(0, 3.141592653589793),(0, 4.5),(0, 6.183185307179587),(1.7, 0),(1.7, 1.7),(1.7, 3.141592653589793),(1.7, 4.5),(1.7, 6.183185307179587),(3.141592653589793, 0),(3.141592653589793, 1.7),(3.141592653589793, 3.141592653589793),(3.141592653589793, 4.5),(3.141592653589793, 6.183185307179587),(4.5, 0),(4.5, 1.7),(4.5, 3.141592653589793),(4.5, 4.5),(4.5, 6.183185307179587),(6.183185307179587, 0),(6.183185307179587, 1.7),(6.183185307179587, 3.141592653589793),(6.183185307179587, 4.5),(6.183185307179587, 6.183185307179587)
